### (1) Load Required Packages

In [2]:
from tensorflow.keras.models import model_from_json
#from source.data_processing import process_data
import numpy as np
import pandas as pd
import glob

### (2) Load Data

In [ ]:
names = ['arthur','Z2RC']
frame = combined_frame('/home/hajar/Dropbox/phd1/CyMET/data/', names)

### (3) Load Immunopred Model

In [5]:
from tensorflow.keras.models import model_from_json #load required packages
json_file = open('/home/hajar/Dropbox/phd1/CyMET/immunopred/model.json', 'r') #open the model
model_tmp = json_file.read() #read the file
json_file.close() #close the file

model = model_from_json(model_tmp) # load model weights
model.load_weights('/home/hajar/Dropbox/phd1/CyMET/immunopred/model_weights.h5')

print("Immunopred Successfully Loaded!")

Immunopred Successfully Loaded!


### (4) Run Cell Prediction

In [ ]:
samp_df = frame.sample(100).reset_index(drop=True)
y_pred = predict_cells(samp_df) #gives you the prediction confidence for each cell type

preds = y_pred.argmax(axis=1) #gives you the actual cell type with highest confidence
samp_df['cell_type'] = preds # map the cell type number to name!

#get the confidence score for final prediction:
scores = []
for x,y in zip(samp_df.index, preds):
    sc = y_pred[x][y] #index and then the position of the highest score
    scores.append(sc)
samp_df['score'] = scores
samp_df

### (5) Summarise predictions at the sample level

In [ ]:
### count the number of healthy and disease samples
hea = []; cov = []; 
for F in samp_df.file.unique():
    if ('COV' in F) or ('_CV' in F) or ('AP' in F) or ('Covid' in F):
        cov.append(F)
    else:
        hea.append(F)
        
print('number of healthy samples: ',len(hea))
print('number of disease samples: ',len(cov))

### append the condition names to each row in the dataframe
nam = []
for F in samp_df.file:
    if F in cov:
        nam.append('covid')
    else:
        nam.append('healthy')

samp_df['state'] = nam
samp_df.head(5)

In [ ]:
### convert data from wide to long format ready for statistical analysis in R
filt = samp_df #[final_df.score >= 0.8] #cant set one filtering threshold, its individual for cells
tokeep = ['cell_type', 'state', 'file']
temp = filt[tokeep].groupby(['file', 'cell_type']).agg(['count'])
temp = temp / temp.groupby(level=0).sum()

## sort out the data in the right format
prop = list(temp.loc[:, 'state']['count'])
file = list(temp.index.get_level_values(0))
nam2 = []
for F in file:
    if F in l2:
        nam2.append('covid')
    else:
        nam2.append('healthy')
pred = list(temp.index.get_level_values(1))
state = nam2

proportion_df = pd.DataFrame(list(zip(prop, file, pred, state)), columns =['len', 'file', 'cluster', 'supp'])
proportion_df.head(10)